# MNIST

In [33]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Denoising AutoEncoder
### EDA

In [34]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [35]:
mnist = keras.datasets.mnist
(x_train, _), (x_test, _) = mnist.load_data()

image_size = x_train.shape[1:]
x_train = np.reshape(x_train, [-1, x_train.shape[1], x_train.shape[2], 1])
x_test = np.reshape(x_test, [-1, x_test.shape[1], x_test.shape[2], 1])
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

# 노이즈 추가
noise = np.random.normal(loc=0.5, scale=0.5, size=x_train.shape)
x_train_noisy = x_train + noise
noise = np.random.normal(loc=0.5, scale=0.5, size=x_test.shape)
x_test_noisy = x_test + noise

# 0. ~ 1. 사이로 제한
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [36]:
x_train.shape

(60000, 28, 28, 1)

### Modeling

In [37]:
from keras.layers import Conv2D, Conv2DTranspose, Flatten, Dense, Reshape
from keras import backend as K

In [38]:
input_shape = x_train.shape[1:]
batch_size = 32
kernel_size = 3
latent_dim = 16
layer_filters = [32, 64]

In [39]:
def encoder(inputs, layer_filters, latent_dim):
    x = inputs
    for filters in layer_filters:
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   activation='relu',
                   strides=2,
                   padding='same'
                  )(x)
    shape = K.int_shape(x) 
    x = Flatten()(x)
    latent = Dense(latent_dim, name='latent_vector')(x)
    
    return latent, shape

In [40]:
inputs = keras.Input(shape=input_shape, name='encoder_input')
latent, shape = encoder(inputs, layer_filters, latent_dim)
encoder = keras.Model(inputs=inputs, outputs=latent, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 flatten_1 (Flatten)         (None, 3136)              0         
                                                                 
 latent_vector (Dense)       (None, 16)                50192     
                                                                 
Total params: 69,008
Trainable params: 69,008
Non-trainable params: 0
_________________________________________________________________


In [46]:
def decoder(latent_inputs, shape, layer_filters, kernel_size):
    x = Dense(shape[1]*shape[2]*shape[3])(latent_inputs) # Flatten
    x = Reshape((shape[1], shape[2], shape[3]))(x)
    
    for filters in layer_filters[::-1]:
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            activation='relu',
                            strides=2,
                            padding='same'
                           )(x)
    outputs = Conv2DTranspose(filters=1,
                              kernel_size=kernel_size,
                              activation='sigmoid',
                              padding='same',
                              name='decoder_output'
                             )(x)

    return outputs

In [47]:
latent_inputs = keras.Input(shape=(latent_dim,), name='decoder_input')
outputs = decoder(latent_inputs, shape, layer_filters, kernel_size)
decoder = keras.Model(inputs=latent_inputs, outputs=outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 decoder_input (InputLayer)  [(None, 16)]              0         
                                                                 
 dense_3 (Dense)             (None, 3136)              53312     
                                                                 
 reshape_3 (Reshape)         (None, 7, 7, 64)          0         
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 14, 14, 64)       36928     
 ranspose)                                                       
                                                                 
 conv2d_transpose_7 (Conv2DT  (None, 28, 28, 32)       18464     
 ranspose)                                                       
                                                                 
 decoder_output (Conv2DTrans  (None, 28, 28, 1)        289 

In [48]:
autoencoder = keras.Model(inputs, decoder(encoder(inputs)), name='autoencoder')

In [49]:
autoencoder.compile(loss='mse',
                    optimizer='adam'
                   )
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder (Functional)        (None, 16)                69008     
                                                                 
 decoder (Functional)        (None, 28, 28, 1)         108993    
                                                                 
Total params: 178,001
Trainable params: 178,001
Non-trainable params: 0
_________________________________________________________________


In [50]:
autoencoder.fit(x_train_noisy, x_train,
                validation_data=(x_test_noisy, x_test),
                epochs=10,
                batch_size=batch_size
               )

Epoch 1/10


2023-02-04 16:17:50.036883: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-04 16:17:50.277335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1873/1875 [============================>.] - ETA: 0s - loss: 0.0349

2023-02-04 16:18:05.842551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 18s 9ms/step - loss: 0.0349 - val_loss: 0.0202
Epoch 2/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0193 - val_loss: 0.0181
Epoch 3/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0177 - val_loss: 0.0172
Epoch 4/10
1875/1875 [==============================] - 16s 8ms/step - loss: 0.0169 - val_loss: 0.0166
Epoch 5/10
1875/1875 [==============================] - 16s 9ms/step - loss: 0.0164 - val_loss: 0.0163
Epoch 6/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0160 - val_loss: 0.0161
Epoch 7/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0157 - val_loss: 0.0160
Epoch 8/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0155 - val_loss: 0.0158
Epoch 9/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0153 - val_loss: 0.0157
Epoch 10/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0152